In [ ]:
# some general notes/TODOs:
# the plan is probably to later turn this into a class that we can construct that would be able to interact with this
# and have a class constant. that's on the list of things to do after the general structure is working (and once I figure 
# out how classes work in python lol)

%matplotlib inline

import time as time

import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering
from sklearn.utils.testing import SkipTest
from sklearn.utils.fixes import sp_version

# this is a ghetto version of saving all the images of individual pieces so we can determine by hand whether 
# or not it's a good piece in order to train the perceptron
# gonna have to figure out a way to actually make sure this number is saved between different runs
img_index = 0

# compression of image width
ZOOM_FACTOR = 25

# input: image array and optional zoom factor z
# output: compressed image array (by factor z^2)
def comp(img,z = ZOOM_FACTOR):
#     img=mpimg.imread(impath)
    compress = img[0:len(img):z,0:len(img[0]):z]
    return compress

# converts image to greyscale by taking averages of rgb values
# returns compressed image, compressed greyscale image
def avg(img, compress = True, z = ZOOM_FACTOR):
    if compress:
        img = comp(img,z)
    w = np.zeros((len(img),len(img[0])))
    for i in range(len(img)):
        for j in range(len(img[0])):
            w[i][j] = int(img[i][j][0]/3.0+img[i][j][1]/3.0+img[i][j][2]/3.0)
    return img, w


# transforms a coordinate from compressed to full size image
def c2f(x,z = ZOOM_FACTOR):
    return int(z * x)


# takes in an image array and optional number of clusters desired
# returns array of size of image whose nonzero values are to which cluster each pixel belongs
def cluster(impath, n = 15, compress = True, usefiltr = False): #usep will filter out small clusters
    full_img = impath
    fig, axes = plt.subplots(1, 2, figsize=(15, 15))
    axes = axes.flatten()

    img, grey = avg(impath, compress)
    axes[0].imshow(img)

    # initializes connectivity matrix
    connectivity = grid_to_graph(*grey.shape)

    X = np.reshape(img, (-1, 3))
    print("Compute structured hierarchical clustering...")
    st = time.time()
    n_clusters = n  # number of regions

    # linkage basically chooses how "distance" between color values computed, ward only setting that worked for me
    ward = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward',
                                   connectivity=connectivity, compute_full_tree = 'auto')
    ward.fit(X)
    label = np.reshape(ward.labels_, grey.shape)
    print("Elapsed time: ", time.time() - st)
    print("Number of pixels: ", label.size)
    print("Number of clusters: ", np.unique(label).size)
    return label
    
# takes in an image array and a clustering label for that image
# optionally takes in whether or not to save the cropped image
# returns a list of areas, perimeters, colors of all pixels in each piece, and colors of all
# background pixels that are in the cropped version together with each piece
def get_cs(impath, label, n_clusters, save_img = false):
    global img_index
    comp_img = comp(impath)
    l = grey.shape[0]
    w = grey.shape[1]
    minp = label.size * MINP
    maxp = label.size * MAXP
    # counts how many pixels are in cluster l
    count = np.zeros((n_clusters + 1,1))
    area = np.zeros((n_clusters + 1,1))
    # this is literally a list of all the values that are in cluster n, to make getting color statistics about it easier
    list_colors = np.zeros((n_clusters + 1, 1))
    list_bcolors = np.zeros((n_clusters + 1, 1))
    dims = np.zeros([n_clusters + 1,4])
    for i in range(len(dims)):
        dims[i][0] = l
        dims[i][2] = w

    # count perimeter of each cluster
    # doesn't count last row/column but that should be okay?
    for i in range(1,np.shape(label)[0]-1):
        for j in range(1,np.shape(label)[1]-1):
            l = label[i,j]
            area[label[i,j]] += 1
            list_colors[label[i,j]].append(comp_img[i,j])
            cl = label[i,j]
            if (cl != label[i,j+1] or cl != label[i, j-1]) or (cl != label[i+1,j] or cl != label[i-1,j]):
                count[label[i,j]] += 1
            # update dim
            if dims[l][0] > i:
                dims[l][0] = i
            if dims[l][1] < i:
                dims[l][1] = i
            if dims[l][2] > j:
                dims[l][2] = j
            if dims[l][3] < j:
                dims[l][3] = j
    
    # can't think of a faster way to do this, so just scan the array again to figure out which background pixels belong to 
    # which cropped image
     for i in range(1,np.shape(label)[0]-1):
        for j in range(1,np.shape(label)[1]-1):
            if (label[i,j] == 0):
                # is a background pixel
                for l in range(1,n_clusters):
                    if (dims[l][0] < i and dims[l][1] > i and dims[l][2] < j and dims[l][3]  > j):
                        # this pixel belongs to the cropped version of this piece
                        list_bcolors[l].append(comp_img[i, j])
    if (save_img):
        # crops and saves the image if desired
        z = ZOOM_FACTOR
        for l in range(1, n_clusters + 1):
            cimg = full_img[c2f(dims[l][0],z)-z:c2f(dims[l][1],z)+z,c2f(dims[l][2],z)-z:c2f(dims[l][3],z)+z]
            imgname = "cropped_piece_" + str(img_index) + ".png"
            img_index += 1
            mpimg.imsave(imgname, cimg)
    return (area, count, list_colors, list_bcolors)
    
# does the full processing on the image
def process_image(impath):
    label = cluster(impath)
    stats = get_cs(impath, label)
    return stats

